In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import torch

In [2]:
from collections import defaultdict

In [3]:
#login to huggingface hub
from huggingface_hub import login
login()

In [4]:
MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct" 

In [5]:
print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", torch_dtype=torch.float16, use_auth_token=True)

Loading model and tokenizer...


/scratch/aseth7/conda-envs/dm-proj/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Step 2: Create a text-generation pipeline
print("Creating text-generation pipeline...")
generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # device=device
)


Creating text-generation pipeline...


In [8]:
print("Loading MMLU dataset...")
dataset = load_dataset("cais/mmlu", "all")

Loading MMLU dataset...


In [ ]:
# def evaluate_on_mmlu(model_pipeline, dataset, num_samples=None, batch_size=8):
#     total_correct = 0
#     total_questions = 0
#     subject_results = defaultdict(lambda: {"correct": 0, "total": 0})
#     csv_results = []

#     print("Evaluating on MMLU...")
#     # Limit dataset size if num_samples is provided
#     if num_samples:
#         dataset['test'] = dataset['test'].select(range(num_samples))

#     for batch_start in range(0, len(dataset['test']), batch_size):
#         batch = dataset['test'].select(range(batch_start, min(batch_start + batch_size, len(dataset['test']))))
#         inputs = []
#         print(f"Processing batch {batch_start + 1} - {batch_start + len(batch)}...")
#         for i in range(len(batch)):
#             question = batch["question"][i]
#             choices = batch["choices"][i]
#             correct_answer = batch["answer"][i]
#             subject = batch["subject"][i]

#             prompt = (
#                 f"Question: {question}\n"
#                 f"Choices: {', '.join(choices)}\n"
#                 "Think step-by-step. Please provide your answer in the format 'Steps: [steps]; Answer: [choice]'."
#             )
#             inputs.append(prompt)

#         # Generate predictions in batch
#         outputs = model_pipeline(
#             inputs, 
#             max_length=2048, 
#             num_return_sequences=1,
#             return_full_text=False
#         )

#         for i, output in enumerate(outputs):
#             question = batch["question"][i]
#             choices = batch["choices"][i]
#             correct_choice = choices[batch["answer"][i]]
#             subject = batch["subject"][i]
#             print(f"\nQuestion: {question}")
#             print(f"Choices: {', '.join(choices)}")
#             print(f"Correct Answer: {correct_choice}")
#             print(f"Prediction: {output[0]['generated_text']}")

#             # Parse the answer from model output
#             print("DEBUG: ", output)
#             prediction = output[0]["generated_text"]
#             if "Answer:" in prediction:
#                 prediction = prediction.split("Answer:")[-1].strip()
#             else:
#                 prediction = ""

#             # Check correctness
#             is_correct = (prediction == correct_choice)
#             if is_correct:
#                 total_correct += 1
#                 subject_results[subject]["correct"] += 1
#             subject_results[subject]["total"] += 1
#             total_questions += 1

#             # Save question result to CSV results
#             csv_results.append({
#                 "subject": subject,
#                 "question": question,
#                 "choices": choices,
#                 "correct_answer": correct_choice,
#                 "prediction": prediction,
#                 "is_correct": is_correct
#             })

#     # Calculate overall accuracy
#     overall_accuracy = total_correct / total_questions if total_questions > 0 else 0

#     # Print per-subject results
#     print("\nPer-Subject Accuracy:")
#     for subject, result in subject_results.items():
#         subject_accuracy = result["correct"] / result["total"] if result["total"] > 0 else 0
#         print(f"{subject}: {subject_accuracy * 100:.2f}%")

#     # Print overall accuracy
#     print(f"\nOverall Accuracy: {overall_accuracy * 100:.2f}%")

#     return overall_accuracy, subject_results, csv_results

In [ ]:
from collections import defaultdict


def format_example(question, choices, include_answer=True, correct_answer=None):
    """
    Format a single question with multiple choices into a prompt template.
    """
    prompt = question
    for idx, choice in enumerate(choices):
        prompt += f"\n{chr(65 + idx)}. {choice}"
    prompt += "\nAnswer:"
    if include_answer and correct_answer is not None:
        prompt += f" {correct_answer}\n"
    return prompt


def gen_prompt(dev_samples, subject):
    """
    Generate a complete prompt for a subject with multiple training examples.
    """
    prompt = f"The following are multiple choice questions (with answers) about {subject}.\n\n"
    for sample in dev_samples:
        question = sample["question"]
        choices = sample["choices"]
        correct_answer = sample["answer"]
        prompt += format_example(question, choices, include_answer=True, correct_answer=correct_answer)
    return prompt


def evaluate_on_mmlu(model_pipeline, dataset, num_samples=None, batch_size=8):
    """
    Evaluate the model on the MMLU dataset using a specific prompt template.
    """
    total_correct = 0
    total_questions = 0
    subject_results = defaultdict(lambda: {"correct": 0, "total": 0})
    csv_results = []

    print("Evaluating on MMLU...")
    # Limit dataset size if num_samples is provided
    if num_samples:
        dataset['test'] = dataset['test'].select(range(num_samples))

    for batch_start in range(0, len(dataset['test']), batch_size):
        batch = dataset['test'].select(range(batch_start, min(batch_start + batch_size, len(dataset['test']))))
        inputs = []
        print(f"Processing batch {batch_start + 1} - {batch_start + len(batch)}...")
        for i in range(len(batch)):
            question = batch["question"][i]
            choices = batch["choices"][i]
            correct_answer_index = batch["answer"][i]
            subject = batch["subject"][i]
            correct_choice = choices[correct_answer_index]

            # Generate the prompt using the new template
            dev_samples = dataset["dev"].filter(lambda ex: ex["subject"] == subject)[:5]  # Few-shot examples
            train_prompt = gen_prompt(dev_samples, subject)
            test_prompt = format_example(question, choices, include_answer=False)
            prompt = train_prompt + test_prompt
            inputs.append(prompt)

        # Generate predictions in batch
        outputs = model_pipeline(
            inputs, 
            max_length=2048, 
            num_return_sequences=1,
            return_full_text=False
        )

        for i, output in enumerate(outputs):
            question = batch["question"][i]
            choices = batch["choices"][i]
            correct_choice = choices[batch["answer"][i]]
            subject = batch["subject"][i]
            print(f"\nQuestion: {question}")
            print(f"Choices: {', '.join(choices)}")
            print(f"Correct Answer: {correct_choice}")
            print(f"Prediction: {output[0]['generated_text']}")

            # Parse the answer from model output
            prediction = output[0]["generated_text"]
            if "Answer:" in prediction:
                prediction = prediction.split("Answer:")[-1].strip()
            else:
                prediction = ""

            # Check correctness
            is_correct = (prediction == correct_choice)
            if is_correct:
                total_correct += 1
                subject_results[subject]["correct"] += 1
            subject_results[subject]["total"] += 1
            total_questions += 1

            # Save question result to CSV results
            csv_results.append({
                "subject": subject,
                "question": question,
                "choices": choices,
                "correct_answer": correct_choice,
                "prediction": prediction,
                "is_correct": is_correct
            })

    # Calculate overall accuracy
    overall_accuracy = total_correct / total_questions if total_questions > 0 else 0

    # Print per-subject results
    print("\nPer-Subject Accuracy:")
    for subject, result in subject_results.items():
        subject_accuracy = result["correct"] / result["total"] if result["total"] > 0 else 0
        print(f"{subject}: {subject_accuracy * 100:.2f}%")

    # Print overall accuracy
    print(f"\nOverall Accuracy: {overall_accuracy * 100:.2f}%")

    return overall_accuracy, subject_results, csv_results

: 

In [47]:
evaluate_on_mmlu(generation_pipeline, dataset, num_samples=1)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Evaluating on MMLU...
Processing batch 1 - 1...

Question: Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.
Choices: 0, 4, 2, 6
Correct Answer: 4
Prediction:  
Step 1:  First, we need to understand the concept of a field extension and its degree. The degree of a field extension L/K is the dimension of L as a vector space over K.
Step 2:  In this problem, we're given the field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q, and we need to find its degree.
Step 3:  To find the degree of the field extension, we start by determining the degrees of the individual extensions Q(sqrt(2))/Q, Q(sqrt(3))/Q, and Q(sqrt(18))/Q.
Step 4:  We know that the degree of Q(sqrt(a))/Q is 2 for any positive integer a not in Q, as the minimal polynomial of sqrt(a) over Q is x^2 - a, which is irreducible over Q.
Step 5:  Now, we need to consider the extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q. Since sqrt(18) = sqrt(9) * sqrt(2) = 3 * sqrt(2), we can express sqrt(18) in ter

(0.0,
 defaultdict(<function __main__.evaluate_on_mmlu.<locals>.<lambda>()>,
             {'abstract_algebra': {'correct': 0, 'total': 1}}),
 [{'subject': 'abstract_algebra',
   'question': 'Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.',
   'choices': ['0', '4', '2', '6'],
   'correct_answer': '4',
   'prediction': '$\\boxed{2}$Steps: Step 1: Understand the concept of a field extension and its degree.Step 2: Find the degrees of the individual extensions Q(sqrt(2))/Q, Q(sqrt(3))/Q, and Q(sqrt(18))/Q.Step 3: Determine the degree of Q(sqrt(2), sqrt(3), sqrt(18))/Q as the LCM of the degrees of the individual extensions Q(sqrt(',
   'is_correct': False}])

In [13]:
for i, sample in enumerate(dataset):
    print(sample)

test
validation
dev
auxiliary_train


In [ ]:
dataset['test'][100]

{'question': 'Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.',
 'subject': 'abstract_algebra',
 'choices': ['0', '4', '2', '6'],
 'answer': 1}